<a href="https://colab.research.google.com/github/espickle1/boltz-2/blob/main/yaml_creator_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Interactive YAML builder (id + sequence ONLY) with correct YAML SPACES (no tabs)
# Prompts for directory + filename; writes ids as one-element lists: id: [A1]

!pip -q install ipywidgets
from google.colab import output, files
output.enable_custom_widget_manager()

import ipywidgets as W
from IPython.display import display, Markdown, clear_output
import os

# ----- protein entry factory: ONLY 'id' and 'sequence' -----
def make_entry():
    id_field = W.Text(placeholder="protein id (e.g., A1)", layout=W.Layout(width="350px"))
    seq_field = W.Textarea(placeholder="protein sequence (AA letters)", layout=W.Layout(width="600px", height="80px"))
    row = W.VBox([
        W.HBox([W.Label("id:", layout=W.Layout(width="60px")), id_field]),
        W.HBox([W.Label("sequence:", layout=W.Layout(width="60px")), seq_field]),
        W.HTML("<hr>")
    ])
    return {"box": row, "id": id_field, "seq": seq_field}

entries = []
def add_entry(_=None):
    e = make_entry()
    entries.append(e)
    entries_box.children = [x["box"] for x in entries]

def remove_last(_=None):
    if entries:
        entries.pop()
        entries_box.children = [x["box"] for x in entries]

entries_box = W.VBox([])
add_btn = W.Button(description="Add protein", button_style="info")
remove_btn = W.Button(description="Remove last")
preview_btn = W.Button(description="Preview YAML", button_style="primary")
save_btn = W.Button(description="Save YAML", button_style="success")
out = W.Output()

# Directory and filename widgets
dir_text = W.Text(value="/content/", description="Directory:", layout=W.Layout(width="500px"))
file_text = W.Text(value="protein.yaml", description="File name:", layout=W.Layout(width="300px"))

add_btn.on_click(add_entry)
remove_btn.on_click(remove_last)

# ---- YAML (spaces, not tabs). Target:
# sequences:
#   - protein:
#       id: [A1]
#       sequence: MDSN...
IND_2 = "  "      # 2 spaces
IND_6 = "      "  # 6 spaces

def normalize_id_to_list(pid: str) -> str:
    pid = pid.strip()
    # If already looks like a YAML list, keep as-is; else wrap in [pid]
    if pid.startswith('[') and pid.endswith(']'):
        return pid
    return f"[{pid}]"

def build_yaml_text():
    lines = ["sequences:"]
    any_item = False
    for e in entries:
        pid = e["id"].value.strip()
        pseq = e["seq"].value.strip()
        if not pid and not pseq:
            continue
        any_item = True
        lines.append(f"{IND_2}- protein:")
        lines.append(f"{IND_6}id: {normalize_id_to_list(pid)}")
        lines.append(f"{IND_6}sequence: {pseq}")
    return ("\n".join(lines) + "\n") if any_item else ""

def on_preview(_):
    with out:
        clear_output()
        y = build_yaml_text()
        if not y:
            display(Markdown("**Add at least one protein entry (id and/or sequence).**"))
            return
        display(Markdown("### YAML Preview"))
        display(Markdown(f"```yaml\n{y}```"))

def on_save(_):
    with out:
        clear_output()
        y = build_yaml_text()
        if not y:
            display(Markdown("**Add at least one protein entry (id and/or sequence).**"))
            return
        directory = dir_text.value.strip()
        fname = file_text.value.strip()
        if not (fname.endswith(".yaml") or fname.endswith(".yml")):
            fname += ".yaml"
        os.makedirs(directory, exist_ok=True)
        full_path = os.path.join(directory, fname)
        with open(full_path, "w", encoding="utf-8") as f:
            f.write(y)
        display(Markdown(f"**Saved:** `{full_path}`"))
        files.download(full_path)

preview_btn.on_click(on_preview)
save_btn.on_click(on_save)

controls = W.HBox([add_btn, remove_btn, preview_btn, save_btn])
display(W.VBox([
    W.HTML("<h3>YAML: sequences → - protein → (id: [ID], sequence)</h3>"),
    dir_text,
    file_text,
    controls,
    entries_box,
    out
]))

# start with one entry
add_entry()
